In [1]:
import numpy as np
import pandas as pd

import matplotlib
import matplotlib.pyplot as plt
matplotlib.style.use('ggplot')
%matplotlib inline


import os

# Подготовим данные

In [2]:
bank = pd.read_csv('bank/bank-full.csv', sep=';', decimal=",")
bank.head()

,age,job,marital,education,default,balance,housing,loan,contact,day,month,duration,campaign,pdays,previous,poutcome,y
0,58,management,married,tertiary,no,2143,yes,no,unknown,5,may,261,1,-1,0,unknown,no
1,44,technician,single,secondary,no,29,yes,no,unknown,5,may,151,1,-1,0,unknown,no
2,33,entrepreneur,married,secondary,no,2,yes,yes,unknown,5,may,76,1,-1,0,unknown,no
3,47,blue-collar,married,unknown,no,1506,yes,no,unknown,5,may,92,1,-1,0,unknown,no
4,33,unknown,single,unknown,no,1,no,no,unknown,5,may,198,1,-1,0,unknown,no


In [3]:
from sklearn.preprocessing import LabelEncoder

labelencoder = LabelEncoder()

bank['job'] = labelencoder.fit_transform(bank['job'])
bank['marital'] = labelencoder.fit_transform(bank['marital'])
bank['education'] = labelencoder.fit_transform(bank['education'])
bank['default'] = labelencoder.fit_transform(bank['default'])
bank['housing'] = labelencoder.fit_transform(bank['housing'])
bank['loan'] = labelencoder.fit_transform(bank['loan'])
bank['contact'] = labelencoder.fit_transform(bank['contact'])
bank['month'] = labelencoder.fit_transform(bank['month'])
bank['poutcome'] = labelencoder.fit_transform(bank['poutcome'])
bank['y'] = labelencoder.fit_transform(bank['y'])

bank.head()

,age,job,marital,education,default,balance,housing,loan,contact,day,month,duration,campaign,pdays,previous,poutcome,y
0,58,4,1,2,0,2143,1,0,2,5,8,261,1,-1,0,3,0
1,44,9,2,1,0,29,1,0,2,5,8,151,1,-1,0,3,0
2,33,2,1,1,0,2,1,1,2,5,8,76,1,-1,0,3,0
3,47,1,1,3,0,1506,1,0,2,5,8,92,1,-1,0,3,0
4,33,11,2,3,0,1,0,0,2,5,8,198,1,-1,0,3,0


In [10]:
X = bank.iloc[:, :-1]
y = bank.iloc[:, 16]

Разделим выборку на обучающую и тестовую (0.33 от всей выборки)

In [11]:
from sklearn.model_selection import train_test_split  
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state = 1234)  

# Строим модель градиентный бустинг

In [14]:
from sklearn.ensemble import GradientBoostingClassifier
from sklearn import metrics

In [15]:
model = GradientBoostingClassifier(random_state=42,
                                   # Доля наблюдений в случайной подвыборке для очередного дерева
                                   subsample=0.66,
                                   # Доля переменных в случайной подвыборке для очередного дерева
                                   max_features='sqrt', 
                                   # Число деревьев
                                   n_estimators=500,
                                   #  критерий качества ‘deviance’ для классификации с вероятностями на выходе
                                   loss='deviance', 
                                   # shrinkage На это число умножаем каждое дерево.
                                   # Рекомендуется выставлять небольшие значения из (0, 0.3].
                                   learning_rate=0.01, 
                                   #  загрязнение дерева измеряем “mse” или “friedman_mse”  (mse с улучшениями) 
                                   criterion='friedman_mse', 
                                   # минимальное уменьшение загрязнения 
                                   min_impurity_decrease=0.001, 
                                   # минимальное число наблюдений в узле потомке
                                   min_samples_leaf=5, 
                                   # минимальное число наблюдений в узле родителе
                                   min_samples_split=10,
                                   # число узлов в дереве
                                   max_depth=5,
                                   verbose=0
                                   )

In [16]:
model.fit(X_train, y_train)

GradientBoostingClassifier(learning_rate=0.01, max_depth=5, max_features='sqrt',
                           min_impurity_decrease=0.001, min_samples_leaf=5,
                           min_samples_split=10, n_estimators=500,
                           random_state=42, subsample=0.66)

In [17]:
y_pred_test = model.predict(X_test)
print(metrics.classification_report(y_test, y_pred_test))

              precision    recall  f1-score   support

           0       0.92      0.97      0.95     13178
           1       0.67      0.40      0.50      1742

    accuracy                           0.91     14920
   macro avg       0.80      0.68      0.72     14920
weighted avg       0.89      0.91      0.90     14920



In [18]:
y_pred_train = model.predict(X_train)
print(metrics.classification_report(y_train, y_pred_train))

              precision    recall  f1-score   support

           0       0.93      0.98      0.95     26744
           1       0.73      0.43      0.55      3547

    accuracy                           0.92     30291
   macro avg       0.83      0.71      0.75     30291
weighted avg       0.91      0.92      0.91     30291



In [21]:
print("Accuracy on train set is ", metrics.accuracy_score(y_train, y_pred_train)*100)
print("Accuracy on test set is ", metrics.accuracy_score(y_test, y_pred_test)*100)

Accuracy on train set is  91.50572777392625
Accuracy on test set is  90.68364611260054


Судя по тому, что на тестовой выборке точность ниже, модель немного переучилась. Ниже подберем параметры получше.

In [19]:
conf_mat = metrics.confusion_matrix(y_test, y_pred_test)
conf_mat = pd.DataFrame(conf_mat, index=model.classes_, columns=model.classes_)
conf_mat

,0,1
0,12841,337
1,1053,689


Первый класс модель прогнозирует все так же плохо. Но лучше, чем моедль Случайный лес.

In [20]:
conf_mat = metrics.confusion_matrix(y_train, y_pred_train)
conf_mat = pd.DataFrame(conf_mat, index=model.classes_, columns=model.classes_)
conf_mat

,0,1
0,26177,567
1,2006,1541


Вывод: пропорциональность сохраняется: доля неправильно определенных классов в обучающей выборке (337, 567) примерно в 2 раза больше, чем в тестовой (1053, 2006), как и количества наблюдений в этих выборках.

In [22]:
fi = pd.DataFrame({'features': X_train.columns, 'importance': model.feature_importances_})
fi.sort_values('importance', ascending=False).head(10)

,features,importance
11,duration,0.442888
10,month,0.106869
15,poutcome,0.106643
13,pdays,0.078582
0,age,0.058304
6,housing,0.049949
8,contact,0.033630
9,day,0.029670
5,balance,0.026373
14,previous,0.024071


Вывод: информативность переменных немного отличается от результатов работы метода случайного леса. Хотя самым информативным (и с большим отрывом) остается параметр duration.

# Подбор гиперпараметров

In [23]:
from sklearn.model_selection import GridSearchCV

Строим сетку гиперпараметров

In [42]:
param_grid = {  
      'n_estimators': [400, 500, 450],
      'min_samples_leaf': [4, 5, 6],
      'max_depth': [4, 5, 6],
      'min_samples_split': [3, 5, 10, 15],
}

Строим модель, параметры которые будем менять

In [43]:
model_2 = GradientBoostingClassifier(random_state=42,
                                   subsample=0.66,
                                   max_features='sqrt', 
                                   n_estimators=500,
                                   loss='deviance', 
                                   learning_rate=0.01, 
                                   criterion='friedman_mse', 
                                   min_impurity_decrease=0.001, 
                                   min_samples_leaf=5, 
                                   min_samples_split=10,
                                   max_depth=5,
                                   verbose=0
                                   )

In [44]:
grid_search_res = GridSearchCV(model_2, param_grid=param_grid, cv=2, 
                               scoring='accuracy', 
                               #  поведение в случае ошибки при вычислении критерия качества
                               error_score='raise', 
                               #  обучить модель с лучшими значениями параметроы
                               refit=True, 
                               pre_dispatch=None, verbose=0)

In [45]:
grid_search_res.fit(X_train, y_train)

GridSearchCV(cv=2, error_score='raise',
             estimator=GradientBoostingClassifier(learning_rate=0.01,
                                                  max_depth=5,
                                                  max_features='sqrt',
                                                  min_impurity_decrease=0.001,
                                                  min_samples_leaf=5,
                                                  min_samples_split=10,
                                                  n_estimators=500,
                                                  random_state=42,
                                                  subsample=0.66),
             param_grid={'max_depth': [4, 5, 6], 'min_samples_leaf': [4, 5, 6],
                         'min_samples_split': [3, 5, 10, 15],
                         'n_estimators': [400, 500, 450]},
             pre_dispatch=None, scoring='accuracy')

Лучший набор значений гиперпараметров из сетки

In [46]:
best_hyperparams = grid_search_res.best_params_
print('Лучшие значения гиперпараметров:\n', best_hyperparams)

Лучшие значения гиперпараметров:
 {'max_depth': 6, 'min_samples_leaf': 6, 'min_samples_split': 15, 'n_estimators': 500}


Лучшее качество на тестовой выборке

In [47]:
best_model = grid_search_res.best_estimator_
test_acc = best_model.score(X_test,y_test)
print('Лучшее accuracy на тестовом множестве', test_acc)

Лучшее accuracy на тестовом множестве 0.9078418230563002


In [48]:
best_model = grid_search_res.best_estimator_
print('Лучшее accuracy на обучающем множестве', best_model.score(X_train,y_train))

Лучшее accuracy на обучающем множестве 0.9234095936086626


Вывод: на обучающей выборке model_2 работает лучше, чем model, а на тестовом качество не сильно отличается от модели с параметрами, подобранными ручным способом

# Построим модель без переменной duration

В описании данных написано, что переменная duration возможно негативно повлияет на результат работы модели. Рассмотрим модель без данной переменной.

In [86]:
bank.head()

,age,job,marital,education,default,balance,housing,loan,contact,day,month,duration,campaign,pdays,previous,poutcome,y
0,58,4,1,2,0,2143,1,0,2,5,8,261,1,-1,0,3,0
1,44,9,2,1,0,29,1,0,2,5,8,151,1,-1,0,3,0
2,33,2,1,1,0,2,1,1,2,5,8,76,1,-1,0,3,0
3,47,1,1,3,0,1506,1,0,2,5,8,92,1,-1,0,3,0
4,33,11,2,3,0,1,0,0,2,5,8,198,1,-1,0,3,0


In [87]:
X2 = bank.iloc[:, :-1].drop('duration', axis=1)
y2 = bank.iloc[:, 16]

In [88]:
X2_train, X2_test, y2_train, y2_test = train_test_split(X2, y2, test_size=0.33, random_state = 1234)  

In [94]:
model_3 = GradientBoostingClassifier(random_state=42,
                                   subsample=0.66,
                                   max_features='sqrt', 
                                   n_estimators=500,
                                   loss='deviance', 
                                   learning_rate=0.01, 
                                   criterion='friedman_mse', 
                                   min_impurity_decrease=0.001, 
                                   min_samples_leaf=6, 
                                   min_samples_split=15,
                                   max_depth=6,
                                   verbose=0
                                   )

In [95]:
model_3.fit(X2_train, y2_train)

GradientBoostingClassifier(learning_rate=0.01, max_depth=6, max_features='sqrt',
                           min_impurity_decrease=0.001, min_samples_leaf=6,
                           min_samples_split=15, n_estimators=500,
                           random_state=42, subsample=0.66)

In [96]:
print("Accuracy on train set is ", metrics.accuracy_score(y2_train, y_pred_train))
print("Accuracy on test set is ", metrics.accuracy_score(y2_test, y_pred_test))

Accuracy on train set is  0.9150572777392625
Accuracy on test set is  0.9068364611260054


Вывод: Качество сильно не изменилось, поэтому можно считать, что параметр сильно на результаты модели не влияет. Хоть и является самым информативным.

In [98]:
fi = pd.DataFrame({'features': X2_train.columns, 'importance': model_3.feature_importances_})
fi.sort_values('importance', ascending=False)

,features,importance
14,poutcome,0.181113
10,month,0.173670
12,pdays,0.124579
0,age,0.116095
6,housing,0.070825
9,day,0.070422
5,balance,0.066284
8,contact,0.052801
13,previous,0.047154
11,campaign,0.029213


Вывод: самый информативным стала переменная poutcome - результат предыдущей маркетинговой кампании. Что очень логично!